SPARK 13 - Obtenga la matriz de distancias en días entre fechas de actualización de las aplicaciones pagas. ¿Cuáles son las dos aplicaciones con mayor distancia? (⭐⭐)

Me piden que agarre las aplicaciones pagas y obtenga la matriz de distancias (en dias) entre las fechas de actualizacion

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [4]:
# Crea la sesion de SPARK
spark = SparkSession.builder.getOrCreate()

# Crea el SPARK content
sc = spark.sparkContext

In [5]:
sqlContext = SQLContext(sc)
infoplay = sqlContext.read.csv("/content/drive/MyDrive/Colab Notebooks/TP1-VISU/googleplaystore.csv", header=True)
inforeviews = sqlContext.read.csv("/content/drive/MyDrive/Colab Notebooks/TP1-VISU/googleplaystore_user_reviews.csv", header=True)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
infoplay.show()

# Queria corroborar cual era la fecha mas pequeña para tener un panorama de cuales serian mas o menos las distancias

In [ ]:
#Fechas mas grande: [Año: 2018, Mes: Agosto, Dia: 8]
infoplay.rdd.filter(lambda x: x['Price'] != '0').map(lambda x: (x['App'] ,tuple(x['Last Updated'].replace(',','').replace(' ','/').split('/'))  )).filter(
    lambda x: len(x[1]) == 3).filter(lambda x: x[1][2] == '2018' and x[1][0] == 'August').distinct().collect()

In [ ]:
#Fechas mas chica: [Año: 2011, Mes: Mayo, Dia: 12]
infoplay.rdd.filter(lambda x: x['Price'] != '0').map(lambda x: (x['App'] ,tuple(x['Last Updated'].replace(',','').replace(' ','/').split('/'))  )).filter(
    lambda x: len(x[1]) == 3).filter(lambda x: x[1][2] == '2011').distinct().collect()

La fecha mas alta fué en 2018 y la mas baja 2011 por lo que el rango de valores será entre 0 y 2606 días aproximadamente

#FuncionesParaFacilitarLaTransformacion

In [9]:
def convertirMesADias(caracter):
  meses = {
      'January': 1,
      'February': 2,
      'March': 3,
      'April': 4,
      'May': 5,
      'June': 6,
      'July': 7,
      'August': 8,
      'September': 9,
      'October': 10,
      'November': 11,
      'December': 12
      }

  return ((meses.get(caracter))*30)

In [10]:
def convertirAñoADias(numero):
  return (int(numero)*360)

In [11]:
def calcularDistanciaEnDias(valor1, valor2):
  return np.linalg.norm(valor1-valor2)

In [12]:
def convertirFechaADias(elemento):
  return (int(convertirMesADias(elemento[0]))+int(elemento[1])+int(convertirAñoADias(elemento[2])))

#OperacionesParaFormarLaMatriz

Filtramos los elementos que sean pagos, luego mapeamos el rdd en una clave (nombre de la app) y una tupla (Mes/Dia/Año de actualizacion). Luego filtramos los que no sean una tupla y eliminamos elementos repetidos.

In [17]:
RDDFiltrado = infoplay.rdd.filter(lambda x: x['Price'] != '0').map(lambda x: (x['App'], tuple(x['Last Updated'].replace(',','').replace(' ','/').split('/'))  )).filter(
    lambda x: len(x[1]) == 3).distinct().map(lambda x: (x[0] , convertirFechaADias(x[1]) )).distinct().cache()

In [ ]:
RDDFiltrado.cartesian(RDDFiltrado).take(10)

Utilizo cartesian() para poder obtener todas las posibles combinaciones entre los elementos de mi RDD.

Luego por medio de map y groupByKey obtengo como clave a cada aplicacion, y como elemento tengo una lista en donde cada posicion es una tupla con el nombre de otra app, y la distancia en dias entre la ultima actualizacion entre ambas app (la app indicada en la clave y la app que se encuentra en esa posicion)

In [18]:
matriz = RDDFiltrado.cartesian(RDDFiltrado).map(lambda x: ( (x[0][0], (x[1][0] ,calcularDistanciaEnDias(int(x[0][1]),int(x[1][1])) )  ))).groupByKey().cache()

In [20]:
matriz.take(10)

[('TurboScan: scan documents and receipts in PDF',
  <pyspark.resultiterable.ResultIterable at 0x798bf55c10c0>),
 ('Tiny Scanner Pro: PDF Doc Scan',
  <pyspark.resultiterable.ResultIterable at 0x798bf55c15d0>),
 ('Puffin Browser Pro',
  <pyspark.resultiterable.ResultIterable at 0x798bf6c67a00>),
 ('Moco+ - Chat, Meet People',
  <pyspark.resultiterable.ResultIterable at 0x798bf55dc040>),
 ('Calculator', <pyspark.resultiterable.ResultIterable at 0x798bf6c67ca0>),
 ('Truth or Dare Pro',
  <pyspark.resultiterable.ResultIterable at 0x798bf568c940>),
 ('Private Dating, Hide App- Blue for PrivacyHider',
  <pyspark.resultiterable.ResultIterable at 0x798bf54dd8a0>),
 ('Ad Blocker for SayHi',
  <pyspark.resultiterable.ResultIterable at 0x798bf55dc070>),
 ('AMBW Dating App: Asian Men Black Women Interracial',
  <pyspark.resultiterable.ResultIterable at 0x798bf6c67bb0>),
 ('Sago Mini Hat Maker',
  <pyspark.resultiterable.ResultIterable at 0x798bf568ca30>)]

#Aplicaciones con mas distancia entre ellas

Para encontrar los dos elementos con mas distancia proponemos que estos sean la distancia del primer elemento con los ultimos dos en actualizarse.

Aplicaciones que fueron las ultimas en actualizarse

In [21]:
RDDFiltrado.top(2, key=lambda x: x[1])

[('Fast Tract Diet', 726728), ('Little Magnet BT Pro', 726725)]

Aplicaciones que fueron las primeras en actualizarse

In [22]:
RDDFiltrado.top(2, key=lambda x: -x[1])

[('AH Alarm Panel', 724122), ('CF-Bench Pro', 724163)]

In [24]:
matriz.filter(lambda x: x[0] == 'AH Alarm Panel').map(lambda x: (x[0], sorted(x[1], key=lambda y: y[1], reverse=True)[:2])).collect()

[('AH Alarm Panel',
  [('Fast Tract Diet', 2606.0), ('Little Magnet BT Pro', 2603.0)])]

Las distancias mas grandes entre fechas de actualizacion son: [2606 dias entre la aplicacion 'AH Alarm Panel' y la aplicacion 'Fast Tract Diet'] y [2603 dias entre la aplicacion 'AH Alarm Panel' y la aplicacion 'Little Magnet BT Pro']